**Agenda**
1. Libraries & Data
2. Initializing Chatbot Training
3. Building the Deep Learning Model
4. Building Chatbot GUI
5. Running Chatbot

### Import Libraries

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /Users/gabii/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/gabii/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Initializing Chatbot Training

In [6]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('Simple-Python-Chatbot/intents.json').read()
intents = json.loads(data_file)

In [7]:
print(data_file)

{"intents": [
        {"tag": "greeting",
         "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],
         "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],
         "context": [""]
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],
         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],
         "context": [""]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me"],
         "responses": ["Happy to help!", "Any time!", "My pleasure"],
         "context": [""]
        },
        {"tag": "noanswer",
         "patterns": [],
         "responses": ["Sorry, can't understand you", "Please give me more info", "Not sure I understand"],
         "context": [""]
        },
        {"tag": "optio

We use the json module to load in the file and save it as the variable intents.

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

If you look carefully at the json file, you can see that there are sub-objects within objects. For example, “patterns” is an attribute within “intents”. So we will use a nested for loop to extract all of the words within “patterns” and add them to our **words** list. We then add to our **documents** list each pair of patterns within their corresponding tag. We also add the tags into our **classes** list, and we use a simple conditional statement to prevent repeats

In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


Next, we will take the **words** list and lemmatize and lowercase all the words inside. In case you don’t already know, **lemmatize** means to turn a word into its base meaning, or its **lemma**. For example, the words “walking”, “walked”, “walks” all have the same lemma, which is just “walk”. The purpose of lemmatizing our words is to narrow everything down to the simplest level it can be. It will save us a lot of time and unnecessary error when we actually process these words for machine learning. This is very similar to **stemming**, which is to reduce an inflected word down to its base or root form.

### Building the Deep Learning Model

In [5]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


Let’s initialize our training data with a variable training. We’re creating a giant nested list which contains bags of words for each of our documents. We have a feature called output_row which simply acts as a key for the list. We then shuffle our training set and do a train-test-split, with the patterns being the X variable and the intents being the Y variable.

In [6]:
#Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
47/47 [==============================] - 3s 56ms/step - loss: 2.2194 - accuracy: 0.1489
Epoch 2/200
47/47 [==============================] - 0s 3ms/step - loss: 2.1094 - accuracy: 0.1702
Epoch 3/200
47/47 [==============================] - 0s 2ms/step - loss: 2.0051 - accuracy: 0.2766
Epoch 4/200
47/47 [==============================] - 0s 5ms/step - loss: 1.9444 - accuracy: 0.3404
Epoch 5/200
47/47 [==============================] - 0s 3ms/step - loss: 1.7206 - accuracy: 0.6170
Epoch 6/200
47/47 [==============================] - 0s 4ms/step - loss: 1.5274 - accuracy: 0.6170
Epoch 7/200
47/47 [==============================] - 0s 3ms/step - loss: 1.4566 - accuracy: 0.6383
Epoch 8/200
47/47 [==============================] - 0s 3ms/step - loss: 1.3721 - accuracy: 0.6170
Epoch 9/200
47/47 [==============================] - 0s 4ms/step - loss: 1.1790 - accur

/usr/local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.100889). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


47/47 [==============================] - 0s 4ms/step - loss: 0.0426 - accuracy: 1.0000
Epoch 96/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0920 - accuracy: 0.9574
Epoch 97/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0759 - accuracy: 0.9787
Epoch 98/200
47/47 [==============================] - 0s 4ms/step - loss: 0.0135 - accuracy: 1.0000
Epoch 99/200
47/47 [==============================] - 0s 4ms/step - loss: 0.0187 - accuracy: 1.0000
Epoch 100/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0202 - accuracy: 1.0000
Epoch 101/200
47/47 [==============================] - ETA: 0s - loss: 0.0532 - accuracy: 0.97 - 0s 2ms/step - loss: 0.0415 - accuracy: 0.9787
Epoch 102/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 1.0000
Epoch 103/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0179 - accuracy: 1.0000
Epoch 104/200
47/47 [==============================] - 0s 5ms/step 

/usr/local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.170981). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


47/47 [==============================] - 0s 4ms/step - loss: 0.0194 - accuracy: 1.0000
Epoch 139/200
47/47 [==============================] - 0s 4ms/step - loss: 0.0514 - accuracy: 0.9787
Epoch 140/200
47/47 [==============================] - 0s 2ms/step - loss: 0.1085 - accuracy: 0.9787
Epoch 141/200
47/47 [==============================] - 0s 4ms/step - loss: 0.0115 - accuracy: 1.0000
Epoch 142/200
47/47 [==============================] - 0s 4ms/step - loss: 0.0126 - accuracy: 1.0000
Epoch 143/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0340 - accuracy: 1.0000
Epoch 144/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0379 - accuracy: 0.9787
Epoch 145/200
47/47 [==============================] - 0s 5ms/step - loss: 0.0106 - accuracy: 1.0000
Epoch 146/200
47/47 [==============================] - 0s 4ms/step - loss: 0.0050 - accuracy: 1.0000 ETA: 0s - loss: 0.0031 - accuracy: 1.0000
Epoch 147/200
47/47 [==============================] - 0s 1ms/s

Now that we have our training and test data ready, we will now use a deep learning model from keras called Sequential. I don’t want to overwhelm you with all of the details about how deep learning models work, but if you are curious, check out the resources at the bottom of the article.
The Sequential model in keras is actually one of the simplest neural networks, a multi-layer perceptron. If you don’t know what that is, I don’t blame you. Here’s the documentation in keras.
This particular network has 3 layers, with the first one having 128 neurons, the second one having 64 neurons, and the third one having the number of intents as the number of neurons. Remember, the point of this network is to be able to predict which intent to choose given some data.
The model will be trained with stochastic gradient descent, which is also a very complicated topic. Stochastic gradient descent is more efficient than normal gradient descent, that’s all you need to know.
After the model is trained, the whole thing is turned into a numpy array and saved as chatbot_model.h5.
We will use this model to form our chatbot interface!

### Building Chatbot GUI

In [7]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('Simple-Python-Chatbot/intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

Once again, we need to extract the information from our files.

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

Here are some functions that contain all of the necessary processes for running the GUI and encapsulates them into units. We have the clean_up_sentence() function which cleans up any sentences that are inputted. This function is used in the bow() function, which takes the sentences that are cleaned up and creates a bag of words that are used for predicting classes (which are based off the results we got from training our model earlier).

In our predict_class() function, we use an error threshold of 0.25 to avoid too much overfitting. This function will output a list of intents and the probabilities, their likelihood of matching the correct intent. The function getResponse() takes the list outputted and checks the json file and outputs the most response with the highest probability.

Finally our chatbot_response() takes in a message (which will be inputted through our chatbot GUI), predicts the class with our predict_class() function, puts the output list into getResponse(), then outputs the response. What we get is the foundation of our chatbot. We can now tell the bot something, and it will then respond back.

In [9]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

Here comes the fun part (if the other parts weren’t fun already). We can create our GUI with tkinter, a Python library that allows us to create custom interfaces.
We create a function called send() which sets up the basic functionality of our chatbot. If the message that we input into the chatbot is not an empty string, the bot will output a response based on our chatbot_response() function.
After this, we build our chat window, our scrollbar, our button for sending messages, and our textbox to create our message. We place all the components on our screen with simple coordinates and heights.

### Areas of Improvement

#### 1. Trying out different neural networks

We used the simplest keras neural network, so there is a LOT of room for improvement. Feel free to try out convolutional networks or recurrent networks for your projects.

#### 2. Using more data
Our json file was extremely tiny in terms of the variety of possible intents and responses. Human language is billions of times more complex than this, so creating JARVIS from scratch will require a lot more.

#### 3. Using different frameworks
There are many more deep learning frameworks than just keras. There’s tensorflow, Apache Spark, PyTorch, Sonnet, and more. Don’t limit yourself to just one tool!

**Resources**
- Deep Learning
- Natural Language Processing
- Command Line
- Neural Networks
- Tkinter

https://towardsdatascience.com/how-to-create-a-chatbot-with-python-deep-learning-in-less-than-an-hour-56a063bdfc44